In [ ]:
# syft absolute
import syft as sy
from syft.service.log.log import SyftLogV3
from syft.types.syft_object import Context
from syft.types.syft_object import SyftObject

TODOS
- [x] action objects
- [x] maybe an example of how to migrate one object type in a custom way
- [x] check SyftObjectRegistry and compare with current implementation
- [x] run unit tests
- [ ] finalize notebooks for testing, run in CI
- [ ] other tasks defined in tickets

In [ ]:
node = sy.orchestra.launch(
    name="test_upgradability",
    dev_mode=True,
    local_db=True,
    n_consumers=2,
    create_producer=True,
    migrate=False,
)

In [ ]:
client = node.login(email="info@openmined.org", password="changethis")

# Client side migrations

## document store objects

In [ ]:
migration_dict = client.services.migration.get_migration_objects()

In [ ]:
def custom_migration_function(context, obj: SyftObject, klass) -> SyftObject:
    # Here, we are just doing the same, but this is where you would write your custom logic
    return obj.migrate_to(klass.__version__, context)

In [ ]:
# this wont work in the cases where the context is actually used,
# but since this would need custom logic here anyway you write workarounds for that (manually querying required state)

context = Context()
migrated_objects = []
for klass, objects in migration_dict.items():
    for obj in objects:
        if isinstance(obj, SyftLogV3):
            migrated_obj = custom_migration_function(context, obj, klass)
        else:
            migrated_obj = obj.migrate_to(klass.__version__, context)
        migrated_objects.append(migrated_obj)

In [ ]:
migrated_objects

In [ ]:
res = client.services.migration.update_migrated_objects(migrated_objects)

In [ ]:
assert isinstance(res, sy.SyftSuccess)

## Actions and ActionObjects

In [ ]:
migration_action_dict = client.services.migration.get_migration_actionobjects()

In [ ]:
# this wont work in the cases where the context is actually used, but since this you would need custom logic here anyway
# it doesnt matter
context = Context()
migrated_actionobjects = []
for klass, objects in migration_action_dict.items():
    for obj in objects:
        # custom migration logic here
        migrated_actionobject = obj.migrate_to(klass.__version__, context)
        migrated_actionobjects.append(migrated_actionobject)

In [ ]:
migrated_actionobjects

In [ ]:
res = client.services.migration.update_migrated_objects(migrated_actionobjects)

In [ ]:
assert isinstance(res, sy.SyftSuccess)